# HPO

In [10]:
%cd ..

/Users/uribagi/Documents/GitHub


In [11]:
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
import optuna
import pennylane as qml
import jax
from jax import numpy as jnp
from utils.nisq import create_grid_connectivity_manual, efficient_connectivity_gates
from datasets.bipartites import BipartiteGraphDataset

In [ ]:
nodes = 10
grid_conn = create_grid_connectivity_manual()
num_qubits = nodes * (nodes - 1) // 2
gates = efficient_connectivity_gates(grid_conn, num_qubits, 1) 
circ = iqp.IqpSimulator(num_qubits, gates, device='lightning.qubit')

train_dataset = BipartiteGraphDataset(num_samples=2048, # Full samples for objective
                                      num_vertices=nodes,
                                      ratio_bipartite=1.0,
                                      edge_prob=0.2)
train_ds, _ = train_dataset.get_all_data() 
train_ds = jnp.array(train_ds)
base_key = jax.random.PRNGKey(42)


Bipartite Dataset (k=10, p=0.2, connected=True):
Target samples: 2048, Generated samples: 2048
Bipartite: 2048, Non-Bipartite: 0
Target Ratio (Bipartite): 1.000, Actual Ratio: 1.000


In [13]:
base_sigma = median_heuristic(train_ds)
base_params_init = initialize_from_data(gates, train_ds)

In [14]:
def objective(trial: optuna.Trial) -> float:
    learning_rate = trial.suggest_float("learning_rate", 3e-5, 0.1, log=True)
    sigma_multiplier = trial.suggest_float("sigma_multiplier", 0.1, 2.0)
    initialization_multiplier = trial.suggest_float("initialization_multiplier", 1e-3, 1.0, log=True)

    trial_key = jax.random.PRNGKey(42 + trial.number)

    sigma = base_sigma * sigma_multiplier
    params_init = base_params_init * initialization_multiplier

    loss_kwarg = {
        "params": params_init,
        "iqp_circuit": circ, 
        "ground_truth": train_ds,
        "sigma": [sigma],
        "n_ops": 1000,   
        "n_samples": 1000, 
        "key": trial_key,  
    }

    loss_function_to_use = genq.mmd_loss_iqp
    trainer = iqp.Trainer("Adam", loss_function_to_use, stepsize=learning_rate)

    n_iters_hpo = 500 

    print(f"\nTrial {trial.number}: LR={learning_rate:.6f}, SigmaM={sigma_multiplier:.4f}, InitM={initialization_multiplier:.6f}")

    try:
        trainer.train(n_iters=n_iters_hpo, loss_kwargs=loss_kwarg, turbo=1)
        final_loss = trainer.losses[-1]

        # Handle NaN or Inf loss values
        if jnp.isnan(final_loss) or jnp.isinf(final_loss) or final_loss > 1e10: # Added high loss check
            print(f"Trial {trial.number} resulted in unstable loss: {final_loss}")
            return float('inf') # Optuna will treat this as a bad trial

        print(f"Trial {trial.number} final loss: {final_loss:.8f}")
        return float(final_loss)

    except Exception as e:
        print(f"Trial {trial.number} failed with exception: {e}")
        return float('inf') # Indicate failure

In [6]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000) 

[I 2025-05-23 09:33:18,835] A new study created in memory with name: no-name-f912c6e1-a55f-43cf-befe-d6b715470541



Trial 0: LR=0.012833, SigmaM=1.3089, InitM=0.030444


Training Progress: 100%|██████████| 500/500 [00:04<00:00, 117.19it/s, loss=0.000002, elapsed time=0.01, total time=4.4]  
[I 2025-05-23 09:33:23,237] Trial 0 finished with value: 1.915236727830007e-06 and parameters: {'learning_rate': 0.012832638548496927, 'sigma_multiplier': 1.3088837497396646, 'initialization_multiplier': 0.03044374039731815}. Best is trial 0 with value: 1.915236727830007e-06.


Training has not converged after 500 steps
Trial 0 final loss: 0.00000192

Trial 1: LR=0.001012, SigmaM=0.8205, InitM=0.003513


Training Progress: 100%|██████████| 500/500 [00:04<00:00, 103.33it/s, loss=0.012731, elapsed time=0.01, total time=4.85]
[I 2025-05-23 09:33:28,089] Trial 1 finished with value: 0.012730698617965091 and parameters: {'learning_rate': 0.0010117441718181656, 'sigma_multiplier': 0.8204903856173144, 'initialization_multiplier': 0.0035127623549693985}. Best is trial 0 with value: 1.915236727830007e-06.


...

Training Progress: 100%|██████████| 500/500 [00:03<00:00, 126.47it/s, loss=0.000001, elapsed time=0.01, total time=3.96] 
[I 2025-05-23 10:33:10,432] Trial 858 finished with value: 6.301993645582958e-07 and parameters: {'learning_rate': 0.011363552176473573, 'sigma_multiplier': 1.9024536866552821, 'initialization_multiplier': 0.06207752783795595}. Best is trial 77 with value: -0.00011012576492267536.


Training has not converged after 500 steps
Trial 858 final loss: 0.00000063

Trial 859: LR=0.013993, SigmaM=1.9684, InitM=0.071275


Training Progress: 100%|██████████| 500/500 [00:03<00:00, 129.11it/s, loss=0.000007, elapsed time=0.01, total time=3.88] 
[I 2025-05-23 10:33:14,326] Trial 859 finished with value: 7.026317608844592e-06 and parameters: {'learning_rate': 0.013993361278104586, 'sigma_multiplier': 1.9683755657705257, 'initialization_multiplier': 0.07127515816238553}. Best is trial 77 with value: -0.00011012576492267536.


Training has not converged after 500 steps
Trial 859 final loss: 0.00000703

Trial 860: LR=0.008392, SigmaM=1.8656, InitM=0.052604


Training Progress:   0%|          | 0/500 [00:00<?, ?it/s, elapsed time=0, loss=0, total time=0]

In [7]:
best_hyperparams = study.best_params
best_loss_value = study.best_value

print("\nOptimization Finished!")
print(f"Best hyperparameters found: {best_hyperparams}")
print(f"Best loss value: {best_loss_value}")


Optimization Finished!
Best hyperparameters found: {'learning_rate': 0.009172089519608987, 'sigma_multiplier': 1.9666857648128913, 'initialization_multiplier': 0.0003543470455562567}
Best loss value: -9.512582234126067e-05


In [ ]:
print("\nRunning full training with best hyperparameters...")

LEARNING_RATE_BEST = best_hyperparams['learning_rate']
SIGMA_MULTIPLIER_BEST = best_hyperparams['sigma_multiplier']
INITIALIZATION_MULTIPLIER_BEST = best_hyperparams['initialization_multiplier']

# Re-initialize components with best hyperparameters
sigma_val = median_heuristic(train_ds) * SIGMA_MULTIPLIER_BEST
params_init_best = initialize_from_data(gates, jnp.array(train_ds)) * INITIALIZATION_MULTIPLIER_BEST

loss_kwarg_best = {
    "params": params_init_best,
    "iqp_circuit": circ,
    "ground_truth": jnp.array(train_ds),
    "sigma": [sigma_val],
    "n_ops": 2000, 
    "n_samples": 2000, 
    "key": base_key, 
}

loss_fn_best = genq.mmd_loss_iqp
trainer_best = iqp.Trainer("Adam", loss_fn_best, stepsize=LEARNING_RATE_BEST)

print(f"Training with: LR={LEARNING_RATE_BEST:.6f}, SigmaM={SIGMA_MULTIPLIER_BEST:.4f}, InitM={INITIALIZATION_MULTIPLIER_BEST:.6f}")
history_best = trainer_best.train(n_iters=2500, loss_kwargs=loss_kwarg_best, turbo=1) # Original iterations
trained_params_best = trainer_best.final_params
final_loss_best_run = trainer_best.losses[-1]

print(f"\nFull training with best params completed.")
print(f"Final loss: {final_loss_best_run:.8f}")


Running full training with best hyperparameters...
Training with: LR=0.009172, SigmaM=1.9667, InitM=0.000354


Training Progress: 100%|██████████| 2500/2500 [00:46<00:00, 53.20it/s, loss=0.000035, elapsed time=0.02, total time=47]   


Training has not converged after 2500 steps

Full training with best params completed.
Final loss: 0.00003535
